#### LangChain文本向量化存储

##### 配置文件准备

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from openai.env file
load_dotenv(".env")

# Read the OPENAI_API_KEY from the environment
api_base = os.getenv("OPENAI_API_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("MODEL_NAME")

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

##### 检查依赖安装

In [3]:
%pip show openai
%pip show langchain
%pip show langchain-openai

Name: openai
Version: 1.14.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: doctran, langchain-openai
Name: langchain
Version: 0.1.12
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental
Name: langchain-openai
Version: 0.0.8
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/lan

##### 词嵌入:将一组文本转换为向量表示

In [2]:
# 使用OpenAI词嵌入模型
from langchain_community.embeddings import OpenAIEmbeddings

# 1.准备文本数据
texts = [
    "你好",
    "你好啊",
    "你叫什么名字?",
    "我叫王大锤",
    "很高兴认识你大锤",
]

# 2.创建词嵌入模型
e_model = OpenAIEmbeddings()

# 生成词嵌入，返回包含对应词嵌入向量的列表
ebeddings = e_model.embed_documents(texts)

# 返回的 ebeddings 变量是一个列表，其中每个元素都是一个 NumPy 数组，表示对应文本的词嵌入向量。
ebeddings

C:\Users\jizhe\AppData\Local\Temp\ipykernel_36612\644848779.py:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  e_model = OpenAIEmbeddings()


APITimeoutError: Request timed out.

##### 向量化的应用场景：语义搜索

In [5]:
# 将文本转换为向量后，可以计算向量之间的相似度，从而找到语义上相似的文本。
embedded_query = e_model.embed_query("这段对话中提到了什么名字?")

# 获取 embedded_query 数组的前五个元素
embedded_query[:5]

[0.004047733608954853,
 0.0009470974041103827,
 0.029676812733286713,
 -0.00633586830824844,
 -0.02480508870743322]

##### 嵌入向量缓存

In [8]:
# 带有缓存功能的嵌入模型
from langchain.embeddings import CacheBackedEmbeddings
# 用于在本地文件系统中存储键值对，用于缓存数据。
from langchain.storage import LocalFileStore
# 用于从文本文件中加载文档
from langchain_community.document_loaders import TextLoader
# 用于按字符分割文本
from langchain.text_splitter import CharacterTextSplitter
# 使用 OpenAI 的嵌入模型
from langchain_community.embeddings import OpenAIEmbeddings

# 1.创建词嵌入模型
u_embeddings = OpenAIEmbeddings()
# 2.创建本地文件存储实例
fs = LocalFileStore("./cache/")
# 3.创建缓存嵌入模型实例
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    u_embeddings,
    fs,
    namespace=u_embeddings.model, # 这将缓存命名空间设置为原始嵌入模型的名称，以避免不同模型之间的缓存冲突。
)

# fs.yield_keys(): 这个方法生成一个包含缓存中所有键的迭代器。
# list(): 这个函数将迭代器转换为列表，以便打印缓存中的所有键。
list(fs.yield_keys())

['text-embedding-ada-002c63ea318-3b5d-533b-960b-46434f8b3c22',
 'text-embedding-ada-002f05b40fb-a095-546e-9c5d-49e069720828',
 'text-embedding-ada-0024250f053-4b1e-5c34-927d-a7857749217f',
 'text-embedding-ada-002b0c54c27-a009-50b4-9ccc-661d5478b195',
 'text-embedding-ada-002e94acbbe-7d17-5331-8310-4e37bdc56d31',
 'text-embedding-ada-0029286d74c-b3fc-56ff-8b08-9071a193f724']

In [9]:
# 加载文档，切分文档

raw_documents = TextLoader("./source/letter.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 610, which is longer than the specified 600


In [ ]:
%pip install faiss-cup

In [10]:
# 将切分文档向量化病存储在缓存中
# 导入FAISS 向量数据库
from langchain_community.vectorstores import FAISS

# %timeit 是 Jupyter Notebook 的魔术命令
# 测量了从文档列表创建 FAISS 索引的耗时
%timeit -r  1 -n 1 db = FAISS.from_documents(documents, cached_embeddings)

74.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
# 查看缓存中的键
list(fs.yield_keys())

['text-embedding-ada-002c63ea318-3b5d-533b-960b-46434f8b3c22',
 'text-embedding-ada-002f05b40fb-a095-546e-9c5d-49e069720828',
 'text-embedding-ada-0024250f053-4b1e-5c34-927d-a7857749217f',
 'text-embedding-ada-002b0c54c27-a009-50b4-9ccc-661d5478b195',
 'text-embedding-ada-002e94acbbe-7d17-5331-8310-4e37bdc56d31',
 'text-embedding-ada-0029286d74c-b3fc-56ff-8b08-9071a193f724']